In [2]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import os
import re
import docx
# import PyPDF2
import io 
import nltk
# import altair as alt
import numpy as np
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import pandas as pd
# import streamlit as st
import numpy as np
# loading the trained model
#pickle_in = open('classifier.pkl', 'rb') 
#classifier = pickle.load(pickle_in)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Win-10\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Win-10\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
import io
import os
import pandas as pd
import  docx2txt
from docx import Document
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import string # special operations on strings
import spacy # language models
from matplotlib.pyplot import imread
from matplotlib import pyplot as plt
from wordcloud import WordCloud
%matplotlib inline
import win32com.client
import warnings
warnings.filterwarnings('ignore')
import textract
import re #regular expression
import string
from nltk.corpus import stopwords
import pprint
from spacy.matcher import Matcher
import multiprocessing as mp
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer


In [13]:

def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub("[0-9" "]+"," ",text)
    text = re.sub('[‘’“”…]', '', text)
    return text

clean = lambda x: clean_text(x)
def cleaned_review(review_df):
    review_df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r","[|]"], value=["","",""], regex=True, inplace=True)
    review_df=review_df.drop(index=[0,1,2,3,4,5,6,7,8,9],axis=0)
    review_df=review_df.reset_index(drop=bool)
    return review_df
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

def extract_skills(nlp_text):
    '''
    Helper function to extract skills from spacy nlp text
    :param nlp_text: object of `spacy.tokens.doc.Doc`
    :param noun_chunks: noun chunks extracted from nlp text
    :return: list of skills extracted
    '''
    tokens = nlp_text
    data = pd.read_csv('E:\DATA SETS\Resumes\Skill.csv')
    skills = list(data.columns.values)
    skillset = []
    # check for one-grams
    for token in tokens:
        if token.lower() in skills:
            skillset.append(token)
    return np.unique(skillset)
    # check for bi-grams and tri-grams
def extract_post(nlp_text):
    '''
    Helper function to extract skills from spacy nlp text
    :param nlp_text: object of `spacy.tokens.doc.Doc`
    :param noun_chunks: noun chunks extracted from nlp text
    :return: list of skills extracted
    '''
    tokens = nlp_text
    data = pd.read_csv('E:\DATA SETS\Resumes\post.csv')
    skills = list(data.columns.values)
    skillset = []
    # check for one-grams
    for token in tokens:
        if token.lower() in skills:
            skillset.append(token)
    return np.unique(skillset)
    # check for bi-grams and tri-grams
def lem(x):
    lmtzr = WordNetLemmatizer()
    lemmatized = [lmtzr.lemmatize(word) for word in x]
    return lemmatized  

def stem(x):
    ps = PorterStemmer()
    stemming = [ps.stem(word) for word in x]
    return stemming   

def extract_designation(x):
    y =''
    for token in x:
        if 'develop' in x and 'sql' in x and 'oracl' in x:
            y ='SQL and Oracle Developer'
        elif 'consult' in x and 'workday' in x:
            y= ' workday consultant'
        elif 'react' in x and 'develop' in x:
             y ='React  Developer'
        elif 'reactjs' in x and 'develop' in x:
             y ='ReactJS  Developer' 
        elif 'develop' in x and 'sql' in x :
            y ='SQL  Developer'
        elif 'hr' in x:
            y = 'human resource'
        elif 'peoplesoft'in x and 'admin' in x:
            y = 'PeopleSoft Admin'
        elif 'peoplesoft' in x and 'consult' in x:
            y ='PeopleSoft Consultant'
        elif token =='softwar':
             y ='Software Engineer'
        elif 'develop' in x and 'sql' in x :
            y ='SQL  Developer' 
        else:
            y ='intern'
    return y
def contactDetails(Text):
    name = ''  
    for i in range(0,3):
        name = " ".join([name, Text[i]])
    return(name)

In [16]:
def resume_classifier(source_directory):
    file_docx = []
    file_path = []
    for file_folder in os.listdir( source_directory):
    #final_path="E:/DATA SETS/Resumes/"+file_folder+".docx"
        final_path = os.path.join( source_directory, file_folder)
        if final_path.endswith('.docx') :
    #for j in os.listdir(os.path.join(r'E:/DATA SETS/Resumes', file_folder)):
        #final_folder_path = os.path.join(final_path,j)
            file_docx.append(textract.process(final_path))
            file_path.append(file_folder)
    df1 = pd.DataFrame(file_docx)
    df1['new_column'] = df1.apply(remove_punctuations)
    df1['Text'] = df1.new_column.str.decode("UTF-8") 
    cleanreviewdf1=cleaned_review(df1)
    df1.Text.drop_duplicates()
    cleaned_df = df1.Text.apply(clean)
    data = pd.concat([df1,cleaned_df],axis = 1)
    stop = stopwords.words('english')
    data['cleaned'] = cleaned_df.apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    df = data['cleaned'].copy()
    df = pd.DataFrame(df)
    
    df['token'] = ''
    j=0
    for i in cleaned_df:
        df.token[j]=i.split()
        j = j+1
        
    f = cleaned_df[0].split()
    
    df['skills'] = ''
    j = 0
    for i in df.token:
        df['skills'][j] = extract_skills(df.token[j])
        j = j+1
    
    df['post'] = ''
    j = 0
    for i in df.token:
        df['post'][j] = extract_post(df.token[j])
        j = j+1
    df['name'] = ''
    j = 0
    for i in df.token:
        df['name'][j] = contactDetails(df.token[j])
        j = j+1    
    
    resume = df.drop(columns={'cleaned','token'}, axis = 1)
    
    resume['lem'] = ''
    j = 0
    for i in resume.post:
        resume['lem'][j] = lem(resume.post[j])
        j = j+1
        
    
    
    resume['stem'] = ''
    j = 0
    for i in resume.lem:
        resume['stem'][j] = stem(resume.lem[j])
        j = j+1
    
    
    resume['designation'] =''
    j = 0
    for i in resume.post:
        resume['designation'][j] = extract_designation(resume.stem[j])
        j = j+1
    
    data = resume[['name','skills','designation']]
    
    
    
    data = pd.concat([data,d],axis=1,ignore_index=False)
    
    return data

In [5]:
import altair as alt
import streamlit as st
import PyPDF2

In [18]:


# this is the main function in which we define our webpage  
def main():

    st.sidebar.header("Resume Classification App")
    st.sidebar.text_input('Enter the resumes folder path')

    # following lines create boxes in which user can enter data required to make prediction 
    #path ='D:/Python_Practice/P-130_Project/Resumes'
    #path = st.sidebar.text_input('Enter the resumes folder path')

    # Create an empty Data Frame ResumeText with two columns
    ResumeText = pd.DataFrame([], columns=['Name', 'skills', 'designation','designation'])
    path = r'E:\DATA SETS\Resumes'
    # when 'Predict' is clicked, make the prediction and store it 
    #if st.button("Upload and Get Result"): 
        # Search the directory path and loop through the resume documents and callthe function getText
    #if st.button("Process"):
#     for filename in os.listdir(path):
#         filename = os.path.join(path, filename)
#         extText = getText(filename)
#         tokText = tokenText(extText)
#         role = roleApplied(tokText)
#         Name = contactDetails(tokText)
#         experience = expDetails(extText)
#         skills = skillSet(tokText)
#         NewRow = [Name,experience, skills,role]
#         ResumeText.loc[len(ResumeText)] = NewRow
#         #st.dataframe(ResumeText)
    ResumeText = resume_classifier(path)
    sqlor = (ResumeText["designation"] == "SQL and Oracle Developer")
        #javares = ResumeText[java]
    workday = (ResumeText["designation"] == "workday consultant")
    react = (ResumeText["designation"] == "React  Developer")
    reactjs = (ResumeText["designation"] == "ReactJS  Developer")
    sql = (ResumeText["designation"] == "SQL  Developer")
    hr = (ResumeText["designation"] == "human resource")
    admin = (ResumeText["designation"] == "PeopleSoft Admin")
    con = (ResumeText["designation"] == "PeopleSoft Consultant")
    soft = (ResumeText["designation"] == "Software Engineer")
    intern = (ResumeText["designation"] == "intern")



    if st.sidebar.button("Display Numbers"):
        st.subheader("No of Resumes Received")
        num = pd.DataFrame(ResumeText[''].value_counts())
        num['designation'] = num.index
        num.set_axis(['No of Resumes', 'designation'], axis='columns', inplace=True)
        num.reset_index(inplace=True, drop=True)
        num = num[['designation', 'No of Resumes']]
        st.dataframe(num)

        base = alt.Chart(num).encode(theta=alt.Theta(
           "No of Resumes:Q", stack=True), color=alt.Color("designation:N", legend=None)
        )
        pie = base.mark_arc(outerRadius=120)
        text = base.mark_text(radius=165, size=15).encode(text="designation:N")
        c = pie + text
        st.altair_chart(c, use_container_width=True)


#     if st.sidebar.button("All Resumes"):
#         st.subheader("ALL RESUMES")
#         exp = st.slider('Select Experience', 0.0, 20.0, 0.1)
#         expres = ResumeText[ResumeText['Exp_years'] >= exp]
#         st.dataframe(expres)

        #st.dataframe(ResumeText)
#     if st.sidebar.button("JAVA Resumes"):
#         st.subheader("JAVA RESUMES")
#         st.dataframe(ResumeText[java])
#     if st.sidebar.button("DBMS Resumes"):
#         st.subheader("DBMS RESUMES")
#         st.dataframe(ResumeText[dbms])
#     if st.sidebar.button("Peoplesoft Resumes"):
#         st.subheader("Peoplesoft Resumes")
#         st.dataframe(ResumeText[peosoft])
#     if st.sidebar.button("Workday Resumes"):
#         st.subheader("Workday Resumes")
#         st.dataframe(ResumeText[workday])


if __name__=='__main__': 
    main()



NameError: name 'd' is not defined